In [1]:
cd D:\InkTop

D:\InkTop


# Import libraries

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
from PIL import Image
import os
import shutil
import pandas as pd
import pytesseract

In [69]:
def crop(img,newspaper,front_page=False):
    '''img is image read with imread'''
    if newspaper=='TOI':
        if front_page:
            image=img[330:-20,10:,:]
        else:
            image=img[120:-20,10:,:]
    elif newspaper=='HT':
        image=img[200:-20,10:,:]
    elif newspaper=='TRIB':
        image=img[230:-20,10:,:]
    return image    

def show(image):
    cv2.namedWindow('win',cv2.WINDOW_NORMAL)
    cv2.imshow('win',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def erosion(image_binary):
    if newspaper=='TOI':
        kernel = np.ones((11,11), np.uint8)
    elif newspaper=='HT':
        kernel=np.ones((9,9), np.uint8)
    elif newspaper=='TRIB':
        kernel=np.ones((9,9), np.uint8)
    erode=cv2.erode(image_binary,kernel)
    return erode

In [70]:
# parameters
newspapers=['TOI','HT','TRIB']
newspaper=newspapers[0]
image_name='TOIDEL_2018_11_04_Page_87.png'
cols=10

In [71]:
ii = cv2.imread(image_name)
# cropping image accordingly
image=crop(ii,newspaper,False)    
# convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
(thresh, image_binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
show(image)

In [72]:
erode=erosion(image_binary)
show(erode)

In [73]:
im2, contours, hierarchy = cv2.findContours(erode,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [74]:
cv2.imwrite('eroded.png',erode)

True

In [75]:
mask=list(hierarchy[0][:,-1]<=0)  #-1
ids=np.arange(len(hierarchy[0]))
id=ids[mask]
new_cnts=[contours[i] for i in id]

In [76]:
#set([cv2.contourArea(i) for i in new_cnts])

In [104]:
carea=[cv2.contourArea(i) for i in new_cnts]
set(carea)

{180.0,
 256.5,
 787.0,
 918.5,
 3235.0,
 3633.5,
 3645.5,
 4097.0,
 9779.5,
 14077.0,
 14134.0,
 14321.0,
 14322.5,
 14324.0,
 14376.5,
 14379.0,
 14380.5,
 14382.0,
 14712.0,
 16879.0,
 17039.0,
 17648.0,
 18359.5,
 18609.5,
 18658.5,
 18839.0,
 19439.5,
 19542.5,
 19618.5,
 19678.0,
 19690.5,
 19717.0,
 19790.0,
 19903.0,
 19986.0,
 20027.5,
 20174.0,
 20228.0,
 20234.0,
 20235.5,
 20259.0,
 20341.0,
 20504.5,
 20591.0,
 20706.5,
 20735.5,
 20745.5,
 20759.0,
 20787.5,
 20801.0,
 20820.5,
 20869.5,
 20895.5,
 20916.0,
 20919.0,
 20924.0,
 20941.0,
 20954.5,
 20968.5,
 20998.5,
 20999.0,
 21000.5,
 21031.5,
 21047.0,
 21053.5,
 21063.5,
 21071.5,
 21086.0,
 21104.0,
 21154.5,
 21155.0,
 21163.0,
 21171.0,
 21184.0,
 21191.5,
 21208.0,
 21214.0,
 21214.5,
 21217.0,
 21227.5,
 21251.0,
 21253.0,
 21271.5,
 21277.5,
 21281.5,
 21287.0,
 21318.5,
 21331.5,
 21332.5,
 21348.0,
 21355.5,
 21383.5,
 21385.5,
 21393.0,
 21401.0,
 21407.5,
 21425.0,
 21431.5,
 21432.5,
 21438.0,
 21439.5,
 21

In [103]:
# depending upon newspaper
if newspaper=='TOI':
    cnts_temp=[i for i in new_cnts if cv2.contourArea(i)>= 10000.0 and cv2.contourArea(i)<=50000.0]
elif newspaper=='HT':
    cnts_temp=[i for i in new_cnts if cv2.contourArea(i)>= 400.0 and cv2.contourArea(i)<=40000.0]
elif newspaper=='TRIB':
    cnts_temp=[i for i in new_cnts if cv2.contourArea(i)>= 18000.0 and cv2.contourArea(i)<=90000.0]    

In [105]:
def x_coord(cont):
    M=cv2.moments(cont)
    return (int(M['m10']/M['m00']))

def y_coord(cont):
    M=cv2.moments(cont)
    return (int(M['m01']/M['m00']))

def check(cont):
    return (cv2.boundingRect(cont)[0])

In [106]:
xlist=np.linspace(0,image.shape[1],cols+1)

In [107]:
#np.argmax(np.logical_xor((check(i)>xlist)[:-1] , check(i)>xlist[1:]))

In [108]:
x_dict={}

for i in range(cols):
    x_dict[i]=[]

for i in cnts_temp:
    val=np.argmax(np.logical_xor((check(i)>xlist)[:-1] , check(i)>xlist[1:]))
    x_dict[val].append(i)
    
for i in x_dict.keys():
    x_dict[i]=sorted(x_dict[i],key=y_coord,reverse=False)

In [109]:
# delete crops folder
shutil.rmtree('./crops/')
#create empty crops folder
os.makedirs('./crops', exist_ok=True)

In [110]:
samp=image.copy()
counter=0
for k in range(cols):
    for sr,i in enumerate(x_dict[k]):
        x,y,w,h=cv2.boundingRect(i)
        cv2.imwrite('crops/'+str(k)+'_'+str(sr)+'_'+str(counter)+'_section.png',image[y:y+h,x:x+w])
        cv2.rectangle(samp,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.putText(samp,str(cv2.contourArea(i)),(x_coord(i),y_coord(i)),cv2.FONT_HERSHEY_DUPLEX,0.5,(255,0,0))
        counter +=1
cv2.imwrite('samp.png',samp)

True

In [102]:
# sort image names
img_loc=os.listdir('crops/')
img_loc.sort(key=lambda x : int(x.split('_')[2])  )


# saving content and writing to csv

In [64]:
heading=''
sub_heading=''
data=[]
for i in img_loc:
    img=cv2.imread('crops/'+i)
    text=pytesseract.pytesseract.image_to_string(img,lang='eng')
    if img.shape[0] <=60:
        if np.mean(img) < 200.0:
            heading=text
        else:
            sub_heading=text
        continue
    data_list=[str(heading),str(sub_heading),str(i.split('_')[1]),str(i.split('_')[0]),str(text)]
    data.append(data_list)
    #data.append(pd.DataFrame([str(heading),str(sub_heading),str(i.split('_')[1]),str(i.split('_')[0]),str(text)]))
    
    #print(data,'\n\n\n\n')

In [65]:
df=pd.DataFrame(columns=['Heading','Sub-Heading','Row','Col','Text_Extracted'],data=data)

In [66]:
df.replace('\n','',inplace=True,regex=True)

In [67]:
df.to_csv('output_'+image_name.split('.')[0]+'.csv', sep=',', encoding='utf-8')

'D:\\InkTop'